# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
###**Updated version by Jorge Carneiro**
Code has been updated based on comments

In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math
import scipy as scp

def convert_hls(image):
    #ADDED
    #converts the image from RGB to HLS
    return cv2.cvtColor(image, cv2.COLOR_RGB2HLS)

def select_rgb_colors(image): 
    #ADDED
    #convert image to HLS
    converted_img = convert_hls(image)
    # define white and yellow thresholds
    white_lower = np.uint8([0, 200, 0])
    white_upper = np.uint8([255, 255, 255])
    yellow_lower = np.uint8([10, 0, 100])
    yellow_upper = np.uint8([40, 255, 255])
    # create white and yellow masks
    white_mask = cv2.inRange(converted_img, white_lower, white_upper)
    yellow_mask = cv2.inRange(converted_img, yellow_lower, yellow_upper)
    # combine to final mask
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    # put mask over base image
    masked = cv2.bitwise_and(image, image, mask = mask)
    return masked

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    #UPDATED in order to allow better reusage of the function
    lines_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
   
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(lines_img, (x1, y1), (x2, y2), color, thickness)
   
    return lines_img

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    #UPDATED simplified in order to allow better reusage of the function
    
    return cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
images = os.listdir("test_images/")


## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.

def road_line_detection(image):
    #ADDED basic detection methods that is just a sequence of the helper functions plus some improvements
    
    #Filters image for white and yellow; from approach 3
    color_masked_img = select_rgb_colors(image)
    #plt.imshow(color_masked_img)
    #plt.show()
    
    #converts to gray scale
    gray_img = grayscale(color_masked_img)
    #plt.imshow(gray_img)
    #plt.show()

    #filters noise; parameter value will define how much detail is blures
    gauss_img = gaussian_blur(gray_img, 15)            
    #plt.imshow(gauss_img)
    #plt.show()

    #edge detection; parameters will dictate which edges will be considered or droped
    canny_img = canny(gauss_img, 50, 150)
    #plt.imshow(canny_img)
    #plt.show()

    #definition of the limits on the image; paramteres will define the shape of the ROI
    ysize = image.shape[0]
    xsize = image.shape[1]
    left_bottom = [0, ysize]
    right_bottom = [xsize, ysize]
    apex = [xsize/2, 4*ysize/7]
    polygon = np.array([ left_bottom, right_bottom, apex ], np.int32)
    limit_img = region_of_interest(canny_img, [polygon])
    #plt.imshow(limit_img)
    #plt.show()

    #preparation of the Hough transform; parameters will dictate what to be considered or droped
    rho = 1
    theta = np.pi/180
    threshold = 20
    min_line_length = 20
    max_line_gap = 300
    
    return hough_lines(limit_img, rho, theta, threshold, min_line_length, max_line_gap)

def basic_lane_detection(image):
    #basic detection method; presents multiple small lines
    detected_lines = road_line_detection(image)
    lines_img = draw_lines(image, detected_lines)
    
    return weighted_img(lines_img,image)

for image_name in images:
    image = mpimg.imread('test_images/'+image_name)
    #plt.imshow(image)
    #plt.show()
    final_img = basic_lane_detection(image)
    plt.imshow(final_img)
    plt.show()
    

In [ ]:
def process_lines(img, lines):
    #advanced detection method using trend lines
    """
    Line processing in order to define one left and one right line based on Hough transform output.
    The logic lies behind the fact that left and right lanes have slopes with oposite signal. Besides that,
    the left lane is most likely situated on ledt side of the picture, the the right lane, on the right side.
    Using that logic, we can remove lines that have a slope/side of the pictures not making sense.
    That way, we can define a reliable trend line, once the strange values have been removed avoiding unwanted results
    """
    x_trendline_l, y_trendline_l, trendline_l, x_trendline_r, y_trendline_r, trendline_r = [], [], [], [], [], []
    #define theoritical frontier between left and right lines
    left_right_frontier=img.shape[1]/2
    #get coordinates values for max and min
    y_size=img.shape[0]
    #print(left_right_frontier)
    #print(lines)
    #print(len(lines))
    #print(lines[0])
    #print(len(lines[0]))
    #print(lines[0][0])
    #print(len(lines[0][0]))
    min_x=min(min(a for [(a,b,c,d)] in lines),min(c for [(a,b,c,d)] in lines))
    max_x=max(max(a for [(a,b,c,d)] in lines),max(c for [(a,b,c,d)] in lines))
    min_y=min(min(b for [(a,b,c,d)] in lines),min(d for [(a,b,c,d)] in lines))
    max_y=max(max(b for [(a,b,c,d)] in lines),max(d for [(a,b,c,d)] in lines))
    for line in lines:
        #analyze each line from the Hough transform; get the slope and the line center to define to which lane line it belongs
        #print(line[0])
        #if line[0][2]  != line[0][0]:
        slope = (line[0][3]-line[0][1])/(line[0][2]-line[0][0])
        line_center = (line[0][2]+line[0][0])/2
        #if 0.5 < abs(slope) < 0.8 :
        if slope < 0 and line_center < left_right_frontier:
            #print('left')
            #negative slope and left side of the image must belong to left lane line
            x_trendline_l.append(line[0][0])  
            x_trendline_l.append(line[0][2])  
            y_trendline_l.append(line[0][1])
            y_trendline_l.append(line[0][3])
        elif slope > 0 and line_center > left_right_frontier:
            #print('right')
            x_trendline_r.append(line[0][0])  
            x_trendline_r.append(line[0][2])  
            y_trendline_r.append(line[0][1])
            y_trendline_r.append(line[0][3])
        #else:
            #unwanted items
            #print('exception')

        #print('for loop')
    
    #print(x_trendline_l)
    #print(len(x_trendline_l))
    #print('####')
    #print(x_trendline_r)
    #print(len(x_trendline_r))
    
    #create the 1st degree trend lines using polyfit
    if len(x_trendline_l) != 0 and len(x_trendline_l) != 0:
        trendline_l = np.polyfit(x_trendline_l, y_trendline_l,1)
    if len(x_trendline_r) != 0 and len(x_trendline_r) != 0:
        trendline_r = np.polyfit(x_trendline_r, y_trendline_r,1)
    
    # convert lines definition from slope and instersection, to coordinates
    xy_l=convert_trendlines(y_size,trendline_l)
    xy_r=convert_trendlines(y_size,trendline_r)
    trendlines=[int(xy_l[0]), int(xy_l[1]), int(xy_l[2]), int(xy_l[3])], [int(xy_r[0]), int(xy_r[1]), int(xy_r[2]), int(xy_r[3])]
    
    return trendlines

def convert_trendlines(y_ini, trendline):
    # convert lines definition from slope and instersection, to coordinates
    y_end=3*y_ini/5
    x_ini=(y_ini-trendline[1])/trendline[0]
    x_end=(y_end-trendline[1])/trendline[0]
    
    return x_ini, y_ini, x_end, y_end

def advanced_lane_detection(image):
    #advanced detection
    detected_lines = road_line_detection(image)
    #adds trend lines creation to the basic method
    trendlines = [process_lines(image, detected_lines)]
    lines_img = draw_lines(image, trendlines,[255,0,0],10)
    
    return weighted_img(lines_img,image)

for image_name in images:
    image = mpimg.imread('test_images/'+image_name)
    final_img = advanced_lane_detection(image)
    plt.imshow(final_img)
    plt.show()
    

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
#import imageio
from moviepy.editor import VideoFileClip
from IPython.display import HTML
#imageio.plugins.ffmpeg.download()

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
#ADDED updated function name to process the images
white_clip = clip1.fl_image(basic_lane_detection) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
#ADDED updated function name to process the images
yellow_clip = clip2.fl_image(advanced_lane_detection)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge_advanced.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
#ADDED updated function name to process the images
challenge_clip = clip3.fl_image(advanced_lane_detection)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))